In [1]:
import pandas as pd
import numpy as np

Merge the price data the same way as the quote and ask data, while for the size variable we sum over each 10 second period.

In [72]:
df_t=pd.read_csv('vwap_test_nktr.csv',header=0)


In [73]:
df_t.head(5)

,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_CORR,TR_SEQNUM,TR_SOURCE,TR_RF
0,20181101,9:30:00.831122376,Q,NKTR,NaN,@O X,12487,38.750,0,15438,N,NaN
1,20181101,9:30:00.831143020,Q,NKTR,NaN,@ Q,12487,38.750,0,15439,N,NaN
2,20181101,9:30:01.237125655,Q,NKTR,NaN,@F I,5,39.000,0,15737,N,NaN
3,20181101,9:30:01.246660171,D,NKTR,NaN,@ I,2,39.320,0,15758,N,Q
4,20181101,9:30:01.905708052,D,NKTR,NaN,@,200,38.725,0,16324,N,Q


In [74]:
D=list(set(df_t['DATE']))

In [75]:

St=[]
for d in D[:-1]:
    s=[]
    i=-1
    T=[]
    df_d=df_t[df_t['DATE']==d]
    for j in range(len(df_d['TIME_M'])):
        if np.array(df_d['TIME_M'])[j][:-13] not in T:
            i+=1
            T.append(np.array(df_d['TIME_M'])[j][:-13])
           
            s.append(np.array(df_d['SIZE'])[j])
        else:
            s[i]+=np.array(df_d['SIZE'])[j]
    St.append(s)
    #print(T)

In [76]:
St=np.array(St,dtype=float)

In [77]:
for i in range(len(St)):
    St[i,:]=St[i,:]/float(np.sum(St[i,:]))

In [78]:
vol_mean=np.mean(St,axis=0)
vol_mean

array([0.15705559, 0.03109365, 0.02890033, 0.0301381 , 0.03292131,
       0.06300413, 0.03857401, 0.02310825, 0.02591751, 0.04282345,
       0.02852976, 0.02826285, 0.0172835 , 0.03185478, 0.04691415,
       0.02330111, 0.03648516, 0.02911683, 0.0237377 , 0.03565267,
       0.0251849 , 0.02283401, 0.02019382, 0.02232226, 0.02394478,
       0.0267878 , 0.02630047, 0.02275722, 0.01703111, 0.01796878])

In [79]:
df_d=df_t[df_t['DATE']==D[-1]]
T=[]
s=[]
p=[]
i=-1
for j in range(len(df_d['TIME_M'])):
    if np.array(df_d['TIME_M'])[j][:-13] not in T:
        i+=1
        T.append(np.array(df_d['TIME_M'])[j][:-13])
        p.append(np.array(df_d['PRICE'])[j])
        s.append(np.array(df_d['SIZE'])[j])
    else:
        s[i]+=np.array(df_d['SIZE'])[j]

In [80]:
s_per=np.array(s)/float(sum(s))

In [84]:
def execution(target_percentage,size,real_volume,real_price):
    i=0
    VWAP=0
    size_=size
    history=[]
    while size>0:
        goal=target_percentage[i]*size_
        if i==len(target_percentage)-1:
            realized=min(size,real_volume[i])
        else:
            realized=min(goal,real_volume[i])
        realized=min(realized,size)
        VWAP+=realized*real_price[i]
        history.append(realized)
        
        i+=1
        size-=realized
    history=np.array(history)
    return history,size,VWAP/float(np.sum(history))
def benchmark(size,real_volume,real_price):
    history=np.array(real_volume)*size/float(sum(real_volume))
    VWAP=np.sum(history*np.array(real_price))
    return history,VWAP/float(np.sum(history))
execution(vol_mean,100000,s,p)

(array([15705.55947455,  3109.36530422,  1234.        ,  3013.80999801,
         3292.13102927,  1501.        ,  3857.4005534 ,  2310.82493492,
         2591.75073673,  4282.34528221,  1800.        ,  2826.28547483,
         1728.34982288,  3185.47838727,  4691.41459274,  2330.11087066,
         3648.51599401,  2911.68292258,  2373.76977447,  3565.26654592,
         2518.48977772,  2283.40078953,  2019.382359  ,  2232.22644479,
         2394.47778552,  2678.77996468,  2630.04740322,  2275.72223135,
         1703.11128825,  9305.30025725]), 0.0, 36.670549392944324)

In [85]:
benchmark(100000,s,p)

(array([7901.1908973 , 1110.42329914,  363.75427426, 3001.12015093,
        2078.4695201 ,  442.45961561, 2718.13465393, 7829.85496993,
        1105.70687419, 5139.42931258,  530.59780686, 2305.15269426,
        7441.04468813, 2757.33993633, 7183.11519868, 2969.28428251,
        3015.56420233, 4058.18889282, 1973.23428841, 3211.0010612 ,
        1468.28204221, 3921.41256927, 1890.40207523,  792.9489447 ,
        1649.56962622, 1340.64379201, 1036.13960618, 8813.8191251 ,
        2786.52281571, 9165.19278387]), 36.59534102493809)